## Import Libraries and Load Dataset



In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

file_path = 'Train.csv'  
df = pd.read_csv(file_path)
df.head()

## Clean Dataset

In [ ]:
df = df.drop_duplicates()
df = df.dropna(subset=['Warehouse_block', 'Mode_of_Shipment'])
df = df.dropna()
df = df.drop_duplicates()
df.head()

## Constructing the Logistics Network

In [ ]:
G = nx.Graph()

for _, row in df[['Warehouse_block', 'Mode_of_Shipment']].iterrows():
    warehouse = row['Warehouse_block']
    shipment_mode = row['Mode_of_Shipment']
    
    if G.has_edge(warehouse, shipment_mode):
        G[warehouse][shipment_mode]['weight'] += 1
    else:
        G.add_edge(warehouse, shipment_mode, weight=1)

print(f'Number of nodes: {G.number_of_nodes()}')
print(f'Number of edges: {G.number_of_edges()}')

## Identifying Important Warehouses

In [ ]:
centrality = {node: sum(weight for _, _, weight in G.edges(node, data='weight')) for node in G.nodes}

warehouse_blocks_centrality = {node: centrality[node] for node in centrality if node in df['Warehouse_block'].values}

top_3_warehouse_blocks = sorted(warehouse_blocks_centrality.items(), key=lambda x: x[1], reverse=True)[:3]

print('Top 3 Most Important Warehouse Blocks:', top_3_warehouse_blocks)

## Visualizing the Logistics Network

In [ ]:
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', 
        node_size=1500, font_size=10)
edge_labels = {(u, v): d['weight'] for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

plt.title('Logistics Network (Warehouses & Shipment Modes)')
plt.show()